# Overview of sugar content of menu items at various restaurants

This project fundamentally relies on data from the [Nutritionix API](http://www.nutritionix.com/api). I am very grateful for the use of their data.

In [3]:
import requests
import json
import os
import minimum_sugar

# Load credential data from file
with open("credentials.json", "r") as f:
    credentials = json.load(f)

## Restaurant IDs
Nutritionix identifies restaurants by a unique number, but from what I can tell they do not list those numbers. The code in this section creates a mapping between the names of restaurants I frequent and Nutritionix's restaurant ID.

In [4]:
restaurant_names = ["McDonalds",
                    "Wendy's",
                    "Taco Bell",
                    "Qdoba",
                    "Chipotle",
                    "Five Guys",]
                    #"Costco",]

restaurant_ids = [minimum_sugar.fetch_restaurant_id(name, credentials) for name in restaurant_names]

In [5]:
# Save the data to a file for reading at a later date.
# with open("restaurant_ids.json", "w") as f:
#     f.write(json.dumps(restaurant_ids, indent=4, separators=(',', ': ')))

## Restaurant menus
Download restaurant menu nutrition data.

In [6]:
# Fetch menu data and add to each dict in `restaurant_ids`
for restaurant in restaurant_ids:
    restaurant["menu"] = minimum_sugar.fetch_menu_item_data(restaurant["id"], credentials)

In [7]:
# Write the data to a file for future analysis
# with open("restaurant_menu_data.json", "w") as f:
#     f.write(json.dumps(restaurant_ids, indent=4, separators=(',', ': ')))

## Misc. observations during development
This section contains some notes on observations I made during development. I intend to include these observations in the report, but rewritten into the body itself and not this section.

First, the legacy/non-fast casual restaurants have a **ton** of menu items. McDonald's has >350 where Chipotle only has like 25. I can't imagine the amout of complexity that number of menu items adds to the management of the company. I also can't see how McDonalds gets rid of this complexity (i.e. sheds menu items) without alienating the customers these items intended to serve. It seems like this amount of complexity is an accretion over many years and is likely a result of their success. It seems eminently plausible that over the years executives at McDonalds thought, "We are dominating this part of the market which is basically tapped out. In order to experience even more growth, we need to expand into other markets. How do we expand into other markets while leveraging the power of this brand to crush the competition?"

Second, executing this project has made me vaguely aware that some of my development practices may not be suited for data science projects. For example, I think the workflow I am using to perform this analysis may not be the most effective. The workflow is: download all data from the server, then write filtering code to eventually get the data I want. I feel like some incarnation of this workflow is what a seasoned data scientist might do, but I suspect most of the filtering will be done by the server or at the database as opposed to at the level of the data scientists local machine.